In [ ]:
project = 'saga-tomtom-prod-wcbc'
use_colab_auth = True

# Legg inn ditt eget prosjekt her, f.eks. 'saga-olanor-playground-ab12'
bq_job_project = ''

In [ ]:
if (use_colab_auth):
  from google.colab import auth
  auth.authenticate_user()
  print('Authenticated')

In [ ]:
import warnings
from google.cloud import bigquery

warnings.filterwarnings('ignore')
client = bigquery.Client(project=bq_job_project)

Denne spørringen gir gjennomsnittlig fart per ukedag i september 2023 for Oslo og Rogaland fylke. Merk at både `segments` og `time_results` bukes i kombinasjon med en kilde til geografidefinisjoner. Dette vil være typisk da TomToms definisjoner av segmenter og deres id er noe man gjerne ikke har kjennskap til.

In [ ]:
query = f"""
-- Hent geometriske definisjonar av dei områda du vil ha med frå ei eigna datakjelde
-- Her brukar vi fylke frå NVDB med Oslo og Rogaland som eksempel
WITH fylker AS (
  SELECT
    fylkesnavn,
    geometri
  FROM `saga-nvdb-prod-vlmh.curated.fylker_med_region`
  WHERE fylkesnavn IN ("Oslo", "Rogaland")
),

-- Finn alle TomTom-segment som ligg innanfor dei områda du henta over
-- Segmenta er unikt definerte av ein segment-id og ein kartversjon
-- Her treng vi óg lengden til kvart av segmenta for å rekne ut vekta summar på tvers av alle segmenta
segments AS (
  SELECT
    segment_id,
    map_version,
    distance_meters, -- segmentlengde i meter
    fylker.fylkesnavn,
FROM `saga-tomtom-prod-wcbc.standardized.segments`
  INNER JOIN fylker
    ON
      ST_CONTAINS (fylker.geometri, geometry)
),

-- For kvart av veisegmenta reknar vi ut ein gjennomsnittleg fart for dei ulike tidsperiodane vi er interesserte i
-- Her ser vi på ulike vekedagar og summerer farta for alle timane i september 2023 som fell på same ukedag, vekta etter antall observasjonar
-- Slik får vi ei fartsmåling per veisegment som er eit gjennomsnitt av alle enkeltobservasjonane på kvar vekedag i løpet av heile månaden
time_results_per_day_of_week AS (
  SELECT
    time_results.segment_id,
    time_results.map_version,
    ANY_VALUE(distance_meters) AS distance_meters,
    ANY_VALUE(fylkesnavn) AS fylkesnavn,
    EXTRACT(DAYOFWEEK FROM sample_date) AS day_of_week,
    SUM(average_speed_kph * sample_size) / SUM(sample_size) AS average_speed
  FROM `saga-tomtom-prod-wcbc.standardized.time_results` AS time_results
  INNER JOIN segments
  ON
    segments.segment_id = time_results.segment_id
    AND segments.map_version = time_results.map_version
  WHERE sample_date BETWEEN "2023-09-01" AND "2023-09-30"
  AND sample_size > 0 -- Det er god praksis å kun nytte timar der det finns observasjonar, sjølv om det ikkje har nokon innverknad på resultatet akkurat her
  GROUP BY segment_id, map_version, day_of_week
)

-- Til slutt summerer vi alle segmenta innanfor kvart av fylka til ei samla gjennomsnittsfart per ukedag
-- Her nyttar vi segmentlengde som vekt for å få ei måling av det aktuelle vegnettet som ikkje avhengjer av korleis TomTom deler vegane inn i segment
-- Om vi ikkje vektar på lengde, vil strekningar med mange og korte segment verke meir inn på resultatet enn tilsvarande strekningar med lengre segment, og det ynskjer vi å unngå
SELECT
  fylkesnavn,
  day_of_week,
  ROUND(SUM(average_speed * distance_meters) / SUM(distance_meters), 2) AS average_speed
FROM time_results_per_day_of_week
GROUP BY fylkesnavn, day_of_week
ORDER BY fylkesnavn, day_of_week
"""

print(query)

client.query(query).to_dataframe()